In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/Code/htfa_rewrite_batching/htfatorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os
import webdataset as wds
import torch

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/'

task_log_csvs = 'editedlogfiles'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub%02d_run%02d_mc_MNI_masked.nii.gz'

AFFVIDS_FILE = 'data/affvids2018_mc_zrest_norest_mini.tar'

tar_file = AFFVIDS_FILE
sink = wds.TarWriter(tar_file)


def affvid_filename(subject, run):
    return affvids_dir + (AFFVID_FILENAME_TEMPLATE % (subject, run))

class TaskElement:
    def __init__(self, task, start, end, run, fear_rating=None):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round(t + TASK_ONSET_DELAY)
                else:
                    return round(t)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)
        self.run = run
        self.fear_rating = fear_rating

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[9])
        end_time = float(cols[10])
        run = int(cols[1])
        fear_rating = float(cols[11])
        yield TaskElement(task, start_time, end_time, run, fear_rating)

def rest_tasks(tasks):
    yield TaskElement('rest', 0, tasks[0].start_time - 1, tasks[0].run)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1
        if tasks[i].run == tasks[i-1].run:
            yield TaskElement('rest', rest_start, rest_end, tasks[i].run)
        else:
            yield TaskElement('rest', rest_start, None, tasks[i-1].run)
            yield TaskElement('rest', 0, rest_end, tasks[i].run)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, tasks[-1].run)

bad_runs = collections.defaultdict(lambda: [])
bad_runs[3] = [1, 2, 3]
bad_runs[7] = [3]
bad_runs[14] = [3]
bad_runs[22] = [1, 2, 3]
bad_runs[23] = [1]
bad_runs[24] = [1, 2, 3]
bad_runs[26] = [3]

## exclude these to create a mini

exclude_subjects = [8, 23, 6, 17, 19, 29, 14, 12, 11, 16, 15, 13, 28, 29, 4, 5, 10, 9]
exclude_tasks = ['rest']

def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0
    with open(task_csv, 'r') as task_csv_file:
        header = task_csv_file.readline().split(' ')
        subject = int(header[4])
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        rest_lines = [r for r in task_lines if r.task == 'rest']
        rest_lines = sorted(rest_lines, key=lambda t: sentinel(t.run))
        rest_starts_dict = {key: [] for key in range(1, 4)}
        rest_ends_dict = {key: [] for key in range(1, 4)}
        for (i,rest) in enumerate(rest_lines):
            if (rest.run in bad_runs[subject]) or (int(subject) in exclude_subjects):
                continue
            if rest.end_time is not None and rest.start_time is not None:
                rest_ends_dict[rest.run].append(rest.end_time)
                rest_starts_dict[rest.run].append(rest.start_time)
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject] or (int(subject) in exclude_subjects) or (task.task in exclude_tasks):
                logging.info('Excluding block %d %s of run %d for motion', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True,zscore_by_rest=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.rest_start_times = rest_starts_dict[result.run]
            result.rest_end_times = rest_ends_dict[result.run]
            result.individual_differences = {'fear_rating': task.fear_rating}
            yield result

In [5]:
OVERRIDE = True

if not os.path.exists(AFFVIDS_FILE) or OVERRIDE:
    total_trs = 0
    metadata = {
        'blocks': []
    }
    block_id = 0
    for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
        for block in read_tasks(task_csv):
            block.filename = affvid_filename(block.subject, block.run)
#             block.mask = mask_filename(block.subject, block.run)
            block.rest_end_times = '[' + ', '.join(map(str, block.rest_end_times)) + ']'
            block.rest_start_times = '[' + ', '.join(map(str, block.rest_start_times)) + ']'
            block.block = block_id
            block_id += 1
            block.mask = '/data/zulqarnain/fmri_data/AffVids_mcwm/wholebrain2.nii.gz'
            block.smooth = 6
            block.load()
            metadata['blocks'].append(block.wds_metadata())

            for vals in block.format_wds():
                sink.write(vals)
            block_trs = (block.end_time - block.start_time)
            total_trs += block_trs
        
#             affvids_db.upsert(block)
    
    metadata['voxel_locations'] = block.locations
    metadata['num_times'] = total_trs
    torch.save(metadata, tar_file + '.meta')
    logging.info('Recorded metadata, including voxel locations')

sink.close()

07/08/2021 10:19:42 Subject 10
07/08/2021 10:19:42 Excluding block 0 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 1 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 2 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 3 spider_low_3.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 4 heights_low_6.m4v of run 3 for motion
07/08/2021 10:19:42 Excluding block 5 spider_high_5.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 6 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 7 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 8 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 9 heights_high_2.mov of run 3 for motion
07/08/2021 10:19:42 Excluding block 10 heights_high_1.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 11 heights_low_1.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 12 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 13 rest of run 2 for motion
07/0

07/08/2021 10:19:42 Excluding block 41 social_low_2.mov of run 3 for motion
07/08/2021 10:19:42 Excluding block 42 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 43 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 44 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 45 social_low_1.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 46 heights_low_4.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 47 social_low_4.m4v of run 3 for motion
07/08/2021 10:19:42 Excluding block 48 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 49 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 50 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 51 social_high_2.mov of run 2 for motion
07/08/2021 10:19:42 Excluding block 52 social_low_5.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 53 spider_low_1.m4v of run 3 for motion
07/08/2021 10:19:42 Excluding block 54 rest of run 2 for motion
07/08/2021 10:19:4

07/08/2021 10:19:42 Excluding block 7 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 8 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 9 spider_high_1.mov of run 3 for motion
07/08/2021 10:19:42 Excluding block 10 spider_low_2.mov of run 1 for motion
07/08/2021 10:19:42 Excluding block 11 social_high_6.mov of run 2 for motion
07/08/2021 10:19:42 Excluding block 12 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 13 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 14 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 15 social_low_1.m4v of run 3 for motion
07/08/2021 10:19:42 Excluding block 16 heights_high_4.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 17 spider_low_6.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 18 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 19 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 20 rest of run 1 for motion
07/08/2021 10:19:42 Excluding b

07/08/2021 10:19:42 Excluding block 48 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 49 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 50 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 51 social_low_5.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 52 heights_low_5.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 53 social_low_4.m4v of run 3 for motion
07/08/2021 10:19:42 Excluding block 54 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 55 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 56 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 57 spider_low_1.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 58 social_low_1.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 59 heights_high_2.mov of run 3 for motion
07/08/2021 10:19:42 Excluding block 60 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 61 rest of run 2 for motion
07/08/2021 10:19:42 Excluding

07/08/2021 10:19:42 Excluding block 15 spider_high_6.mov of run 3 for motion
07/08/2021 10:19:42 Excluding block 16 social_high_1.mov of run 1 for motion
07/08/2021 10:19:42 Excluding block 17 heights_high_2.mov of run 2 for motion
07/08/2021 10:19:42 Excluding block 18 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 19 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 20 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 21 heights_high_1.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 22 social_low_4.m4v of run 3 for motion
07/08/2021 10:19:42 Excluding block 23 spider_low_1.m4v of run 2 for motion
07/08/2021 10:19:42 Excluding block 24 rest of run 1 for motion
07/08/2021 10:19:42 Excluding block 25 rest of run 3 for motion
07/08/2021 10:19:42 Excluding block 26 rest of run 2 for motion
07/08/2021 10:19:42 Excluding block 27 spider_high_2.m4v of run 1 for motion
07/08/2021 10:19:42 Excluding block 28 heights_low_3.mov of run 3 for motion


...

07/08/2021 10:21:09 Excluding block 23 spider_low_1.m4v of run 3 for motion
07/08/2021 10:21:09 Excluding block 24 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 25 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 26 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 27 spider_high_6.mov of run 1 for motion
07/08/2021 10:21:09 Excluding block 28 heights_low_3.mov of run 2 for motion
07/08/2021 10:21:09 Excluding block 29 spider_low_4.m4v of run 3 for motion
07/08/2021 10:21:09 Excluding block 30 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 31 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 32 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 33 spider_high_1.mov of run 2 for motion
07/08/2021 10:21:09 Excluding block 34 heights_high_3.m4v of run 1 for motion
07/08/2021 10:21:09 Excluding block 35 spider_high_5.m4v of run 3 for motion
07/08/2021 10:21:09 Excluding block 36 rest of run 2 for motion
07/08/2021 10:

07/08/2021 10:21:09 Excluding block 65 social_high_1.mov of run 3 for motion
07/08/2021 10:21:09 Excluding block 66 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 67 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 68 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 69 social_high_5.m4v of run 1 for motion
07/08/2021 10:21:09 Excluding block 70 social_low_3.m4v of run 2 for motion
07/08/2021 10:21:09 Excluding block 71 social_low_5.m4v of run 3 for motion
07/08/2021 10:21:09 Excluding block 72 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 73 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 74 rest of run 3 for motion
07/08/2021 10:21:09 Subject 5
07/08/2021 10:21:09 Excluding block 0 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 1 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 2 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 3 heights_high_1.m4v of run 1 for motion
07/08/2021 10:

07/08/2021 10:21:09 Excluding block 31 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 32 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 33 social_high_2.mov of run 1 for motion
07/08/2021 10:21:09 Excluding block 34 spider_low_2.mov of run 2 for motion
07/08/2021 10:21:09 Excluding block 35 heights_high_3.m4v of run 3 for motion
07/08/2021 10:21:09 Excluding block 36 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 37 rest of run 2 for motion
07/08/2021 10:21:09 Excluding block 38 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 39 spider_high_5.m4v of run 1 for motion
07/08/2021 10:21:09 Excluding block 40 heights_low_2.m4v of run 3 for motion
07/08/2021 10:21:09 Excluding block 41 spider_high_2.m4v of run 2 for motion
07/08/2021 10:21:09 Excluding block 42 rest of run 3 for motion
07/08/2021 10:21:09 Excluding block 43 rest of run 1 for motion
07/08/2021 10:21:09 Excluding block 44 rest of run 2 for motion
07/08/2021 10:21:09 Exclud

07/08/2021 10:21:31 Excluding block 60 rest of run 1 for motion
07/08/2021 10:21:31 Excluding block 61 rest of run 3 for motion
07/08/2021 10:21:31 Excluding block 62 rest of run 2 for motion
07/08/2021 10:21:31 Block 63 social_high_1.mov of run 1 started at 458.000000, ended at 478.000000
07/08/2021 10:21:31 Excluding block 64 heights_high_2.mov of run 3 for motion
07/08/2021 10:21:31 Block 65 social_high_4_replacement.mov of run 2 started at 465.000000, ended at 485.000000
07/08/2021 10:21:31 Excluding block 66 rest of run 1 for motion
07/08/2021 10:21:31 Excluding block 67 rest of run 3 for motion
07/08/2021 10:21:31 Excluding block 68 rest of run 2 for motion
07/08/2021 10:21:31 Block 69 social_high_2.mov of run 1 started at 502.000000, ended at 523.000000
07/08/2021 10:21:31 Excluding block 70 heights_low_5.m4v of run 3 for motion
07/08/2021 10:21:31 Block 71 social_high_5.m4v of run 2 started at 508.000000, ended at 528.000000
07/08/2021 10:21:31 Excluding block 72 rest of run 1 

07/08/2021 10:21:31 Excluding block 25 rest of run 3 for motion
07/08/2021 10:21:31 Excluding block 26 rest of run 1 for motion
07/08/2021 10:21:31 Excluding block 27 spider_high_2.m4v of run 2 for motion
07/08/2021 10:21:31 Excluding block 28 spider_low_3.m4v of run 3 for motion
07/08/2021 10:21:31 Excluding block 29 spider_high_6.mov of run 1 for motion
07/08/2021 10:21:31 Excluding block 30 rest of run 2 for motion
07/08/2021 10:21:31 Excluding block 31 rest of run 1 for motion
07/08/2021 10:21:31 Excluding block 32 rest of run 3 for motion
07/08/2021 10:21:31 Excluding block 33 social_low_1.m4v of run 2 for motion
07/08/2021 10:21:31 Excluding block 34 heights_low_1.m4v of run 3 for motion
07/08/2021 10:21:31 Excluding block 35 social_high_5.m4v of run 1 for motion
07/08/2021 10:21:31 Excluding block 36 rest of run 2 for motion
07/08/2021 10:21:31 Excluding block 37 rest of run 3 for motion
07/08/2021 10:21:31 Excluding block 38 rest of run 1 for motion
07/08/2021 10:21:31 Excludin

In [6]:
logging.info('Finished building NiiDb out of AffVids dataset in %s', affvids_dir)

07/08/2021 10:21:31 Finished building NiiDb out of AffVids dataset in /data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/
